<a href="https://colab.research.google.com/github/Vanbk/Tensorflow/blob/master/cnn_raw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

tf.reset_default_graph()

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
# Traning parameter
learning_rate = 0.001
num_steps = 1000
batch_size = 128
display_step = 10

# Netework Parameter

num_input = 784 # MNIST data input
num_classes = 10 # Mnist total class
dropout = 0.75 # Dropout, probability to keep units

# tf Graph
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [0]:
# Create some wrappers for similarity

def conv2d(x, W, b, strides = 1):
  x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding= 'SAME')
  x = tf.nn.bias_add(x, b)
  return tf.nn.relu(x)

def maxpool2d(x, k = 2):
  return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides = [1, k, k, 1],
                        padding = 'SAME')

def conv_net(x, weights, biases, dropout):
  x = tf.reshape(x, shape = [-1, 28, 28, 1])
  # conv1
  conv1 = conv2d(x, weights['wc1'], biases['bc1'])
  conv1 = maxpool2d(conv1, k =2)

  #conv2
  conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
  conv2 = maxpool2d(conv2, k = 2)

  # fully connected layer
  # Reshape conv2 to fit fc layer
  fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
  fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
  fc1 = tf.nn.relu(fc1)

  # dropout layer

  fc1 = tf.nn.dropout(fc1, dropout)
  # output, class prediction

  out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])

  return out

In [0]:
weights = {
    # filter_heigh = 5, filter_shape = 5, input_channel = 1, output_channel= 32
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # filter_heigh = 5, filter_shape = 5, input_channel = 32, output_channel= 64
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),

    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [0]:
# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# loss and optimizer

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                                  labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)

#Evaluate model
correct_pre = tf.equal(tf.arg_max(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pre, tf.float32))

# initialize 
init = tf.global_variables_initializer()

In [0]:
with tf.Session() as sess:
  # Init the variable
  sess.run(init)
  for step in range (1, num_steps + 1):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    sess.run(optimizer, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})

    if step % display_step == 0 or step == 1:
      train_loss, train_acc = sess.run([loss_op, accuracy], 
                  feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0})
      
      print ("Step: {} , train_loss: {}, accuracy: {}".format(step, train_loss, train_acc))

Step: 1 , train_loss: 48551.27734375, accuracy: 0.140625
Step: 10 , train_loss: 20307.171875, accuracy: 0.21875
Step: 20 , train_loss: 11193.8203125, accuracy: 0.4609375
Step: 30 , train_loss: 6568.57958984375, accuracy: 0.6484375
Step: 40 , train_loss: 4051.62451171875, accuracy: 0.703125
Step: 50 , train_loss: 3457.119873046875, accuracy: 0.8203125
Step: 60 , train_loss: 3486.64111328125, accuracy: 0.8046875
Step: 70 , train_loss: 3667.2841796875, accuracy: 0.7890625
Step: 80 , train_loss: 2564.18212890625, accuracy: 0.8671875
Step: 90 , train_loss: 2275.117919921875, accuracy: 0.84375
Step: 100 , train_loss: 1155.3125, accuracy: 0.8984375
Step: 110 , train_loss: 1503.26171875, accuracy: 0.8984375
Step: 120 , train_loss: 1504.989990234375, accuracy: 0.8984375
Step: 130 , train_loss: 1521.0225830078125, accuracy: 0.90625
Step: 140 , train_loss: 965.5655517578125, accuracy: 0.9140625
Step: 150 , train_loss: 801.234130859375, accuracy: 0.9375
Step: 160 , train_loss: 731.9411010742188, a